In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import babar_analysis_tools as bat

from analysis_variables import *

import myPIDselector

import pandas as pd
import seaborn as sns

In [ ]:
#region_definitions

In [ ]:
#####################################################################
# Where are we running this?
#####################################################################
## Bellis computer
topdir= "/home/bellis/babar_data_local/bnv_plambda"

## My laptop
#topdir= "/Users/josieswann/BaBar_analyses/BNV_pLambda/"
#####################################################################


#####################################################################
# Get the BNV data
#####################################################################
data, data_collision = bat.load_datasets(topdir=topdir, subset='Run1')

#####################################################################
# Get the BNC data
#####################################################################
#topdir= "/home/bellis/babar_data/bnv_plambda_bnc"
#data, data_collision = bat.load_datasets(topdir=topdir, BNC=True, subset='all')
#data, data_collision = bat.load_datasets(topdir=topdir, BNC=True, subset='Run1')

# Cut out duplicates

mask_dup = bat.get_duplicates_mask(data)
data_temp = data[mask_dup]

data = data_temp

In [ ]:
savedir = './BNV_pLambda_plots/'

def plot_variables_for_lambda0_purity_study(data, spmode='-999', tag='DEFAULT', region_definitions=None):
    
    mask = data['spmode'] == spmode
    
    mask_fit = None
    if region_definitions is not None:
        mes = data['BpostFitMes']
        de = data['BpostFitDeltaE']

        meslo = region_definitions['fitting MES'][0]
        meshi = region_definitions['fitting MES'][1]
        DeltaElo = region_definitions['fitting DeltaE'][0]
        DeltaEhi = region_definitions['fitting DeltaE'][1]

        mask_fit = (mes>meslo) & (mes<meshi) & (de>DeltaElo) & (de<DeltaEhi)
        #mask = mask & mask_fit
        mask_fit = mask_fit[mask]
        
    #print(data[mask]['B_con_Vtxx'][mask_fit])
    
    # Pull out all the data we might need
    bvtx = data[mask]['B_con_Vtxx'][mask_fit]
    bvty = data[mask]['B_con_Vtxy'][mask_fit]
    bvtz = data[mask]['B_con_Vtxz'][mask_fit]
    
    lamvtx = data[mask]['Lambda0Vtxx'][mask_fit]
    lamvty = data[mask]['Lambda0Vtxy'][mask_fit]
    lamvtz = data[mask]['Lambda0Vtxz'][mask_fit]

    lamfl_basic =    data[mask]['Lambda0FlightLen'][mask_fit]
    
    lamconfl =    data[mask]['Lambda0postFitFlight'][mask_fit]
    lamconflsig = data[mask]['Lambda0postFitFlightSignificance'][mask_fit]
    
    bconfl =    data[mask]['BpostFitFlight'][mask_fit]
    bconflsig = data[mask]['BpostFitFlightSignificance'][mask_fit]
    
    blamfldx = bvtx - lamvtx
    blamfldy = bvty - lamvty
    
    blamfltrans2 = blamfldx**2 + blamfldx**2
    blamfltrans = np.sqrt(blamfltrans2)
    
    lamfltrans2 = lamvtx**2 + lamvtx**2
    lamfltrans = np.sqrt(lamfltrans2)

    lamuncmass = data[mask]['Lambda0_unc_Mass'][mask_fit]

    # Draw some plots of the original distributions
    plt.figure()
    plt.hist(ak.flatten(lamuncmass),bins=100)
    plt.xlabel(f'$\Lambda^0$ mass (GeV/c$^2$)', fontsize=18)
    outfile = f'{savedir}/STUDY_flight_len_effect_on_Lambda_purity_Lambda_unc_mass_{spmode}.png'
    plt.savefig(outfile)

    # Flight length variables

    plt.figure()    
    plt.hist(ak.flatten(lamconfl),bins=100, range=(0,60))
    plt.xlabel(f'$\Lambda^0$ post-fit flight length (cm)')

    outfile = f'{savedir}/STUDY_flight_len_effect_on_Lambda_purity_Lambda_postfit_flightlen_{spmode}.png'
    plt.savefig(outfile)

    ###################################################
    
    plt.figure(figsize=(8,8))
    
    plt.subplot(2,2,1)
    plt.hist(ak.flatten(lamconfl),bins=100, range=(0,60))
    plt.xlabel(f'$\Lambda^0$ post-fit flight length (cm)')
    
    plt.subplot(2,2,2)
    plt.hist(ak.flatten(lamconflsig),bins=100, range=(0,300))
    plt.xlabel(f'$\Lambda^0$ post-fit flight length significance')
    
    plt.subplot(2,2,3)
    plt.hist(ak.flatten(bconfl),bins=100, range=(0,.2))
    plt.xlabel(f'$B$ post-fit flight length (cm)')
    
    plt.subplot(2,2,4)
    plt.hist(ak.flatten(bconflsig),bins=100, range=(0,4))
    plt.xlabel(f'$B$ post-fit flight length significance')

    #plt.subplot(3,2,5)
    #plt.hist(ak.flatten(lamfl_basic),bins=100, range=(0,4))
    #plt.xlabel(f'$\Lambda^0$ flight length (basic)')
    #plt.tight_layout()

    ########################################

    plt.figure(figsize=(12,12))
    
    plt.subplot(4,3,1)
    plt.hist(ak.flatten(lamconfl),bins=100, range=(0,60))
    plt.xlabel(f'$\Lambda^0$ post-fit flight length (cm)')
    plt.subplot(4,3,2)
    plt.hist(ak.flatten(lamconfl),bins=100, range=(0,10))
    plt.xlabel(f'$\Lambda^0$ post-fit flight length (cm)')
    plt.subplot(4,3,3)
    plt.hist(ak.flatten(lamconfl),bins=100, range=(0,2))
    plt.xlabel(f'$\Lambda^0$ post-fit flight length (cm)')

    
    plt.subplot(4,3,4)
    plt.hist(ak.flatten(lamconflsig),bins=100, range=(0,300))
    plt.xlabel(f'$\Lambda^0$ post-fit flight length significance')
    plt.subplot(4,3,5)
    plt.hist(ak.flatten(lamconflsig),bins=100, range=(0,50))
    plt.xlabel(f'$\Lambda^0$ post-fit flight length significance')
    plt.subplot(4,3,6)
    plt.hist(ak.flatten(lamconflsig),bins=100, range=(0,10))
    plt.xlabel(f'$\Lambda^0$ post-fit flight length significance')
    
    plt.subplot(4,3,7)
    plt.hist(ak.flatten(bconfl),bins=100, range=(0,.2))
    plt.xlabel(f'$B$ post-fit flight length (cm)')
    plt.subplot(4,3,8)
    plt.hist(ak.flatten(bconfl),bins=100, range=(0,.05))
    plt.xlabel(f'$B$ post-fit flight length (cm)')
    plt.subplot(4,3,9)
    plt.hist(ak.flatten(bconfl),bins=100, range=(0,.01))
    plt.xlabel(f'$B$ post-fit flight length (cm)')
    
    plt.subplot(4,3,10)
    plt.hist(ak.flatten(blamfltrans),bins=100, range=(0,60))
    plt.xlabel(f'$B-\Lambda^0$ transverse flight length (cm)')
    plt.subplot(4,3,11)
    plt.hist(ak.flatten(blamfltrans),bins=100, range=(0,5))
    plt.xlabel(f'$B-\Lambda^0$ transverse flight length (cm)')
    plt.subplot(4,3,12)
    plt.hist(ak.flatten(blamfltrans),bins=100, range=(0,1))
    plt.xlabel(f'$B-\Lambda^0$ transverse flight length (cm)')

    
    plt.tight_layout()
    outfile = f'{savedir}/STUDY_flight_len_Lambda_variables_zoomed_in_{spmode}.png'
    plt.savefig(outfile)

    ########################################
    
    # Transverse len
    plt.figure(figsize=(12,4))
    
    plt.subplot(1,3,1)
    plt.hist(ak.flatten(lamconfl),bins=100, range=(0,30))
    plt.xlabel(f'$\Lambda^0$ post-fit flight length (cm)')
    
    plt.subplot(1,3,2)
    plt.hist(ak.flatten(blamfltrans),bins=100, range=(0,5))
    plt.xlabel(f'$B-\Lambda^0$ transverse flight length (cm)')
    
    plt.subplot(1,3,3)
    plt.hist(ak.flatten(lamfltrans),bins=100, range=(0,5))
    plt.xlabel(f'$\Lambda^0$ transverse flight length (cm)')

    plt.tight_layout()
    
    outfile = f'{savedir}/STUDY_subset_of_variables_used_in_Lambda_purity_study_{spmode}.png'
    plt.savefig(outfile)

    return 0

###########################################################################################


In [ ]:
plot_variables_for_lambda0_purity_study(data, spmode='998', region_definitions=region_definitions)

In [ ]:
plot_variables_for_lambda0_purity_study(data, spmode='-999')

In [ ]:
plot_variables_for_lambda0_purity_study(data, spmode='1005')

In [ ]:
def study_flight_length_effect_on_lambda0_purity(data, spmode='-999', tag='DEFAULT', region_definitions=None):
    
    mask = data['spmode'] == spmode
    
    mask_fit = None
    if region_definitions is not None:
        mes = data['BpostFitMes']
        de = data['BpostFitDeltaE']

        meslo = region_definitions['fitting MES'][0]
        meshi = region_definitions['fitting MES'][1]
        DeltaElo = region_definitions['fitting DeltaE'][0]
        DeltaEhi = region_definitions['fitting DeltaE'][1]

        mask_fit = (mes>meslo) & (mes<meshi) & (de>DeltaElo) & (de<DeltaEhi)
        #mask = mask & mask_fit
        mask_fit = mask_fit[mask]

    # Pull out all the data we might need
    bvtx = data[mask]['B_con_Vtxx'][mask_fit]
    bvty = data[mask]['B_con_Vtxy'][mask_fit]
    bvtz = data[mask]['B_con_Vtxz'][mask_fit]
    
    lamvtx = data[mask]['Lambda0Vtxx'][mask_fit]
    lamvty = data[mask]['Lambda0Vtxy'][mask_fit]
    lamvtz = data[mask]['Lambda0Vtxz'][mask_fit]

    lamfl_basic =    data[mask]['Lambda0FlightLen'][mask_fit]
    
    lamconfl =    data[mask]['Lambda0postFitFlight'][mask_fit]
    lamconflsig = data[mask]['Lambda0postFitFlightSignificance'][mask_fit]
    
    bconfl =    data[mask]['BpostFitFlight'][mask_fit]
    bconflsig = data[mask]['BpostFitFlightSignificance'][mask_fit]
    
    blamfldx = bvtx - lamvtx
    blamfldy = bvty - lamvty
    
    blamfltrans2 = blamfldx**2 + blamfldx**2
    blamfltrans = np.sqrt(blamfltrans2)
    
    lamfltrans2 = lamvtx**2 + lamvtx**2
    lamfltrans = np.sqrt(lamfltrans2)

    lamuncmass = data[mask]['Lambda0_unc_Mass'][mask_fit]
    
    #fig, axes = plt.subplots(ncols=2, figsize=(8,4))

    #print(fig1)
    #print(axes1)
    
    peaks = []
    allentries = []
    sigs = []
    bkgs = []
    pcts = []
    cuts = []
    tags = []

    # Define the mass cuts around the lambda
    lammass_world_average = 1.115683
    width = 0.003 # GeV/c^2
    
    lo = lammass_world_average - width
    hi = lammass_world_average + width

    #flight_variables = [blamfltrans, lamconfl, lamfl_basic, lamconflsig]
    #flight_tags = ['blamtrans', 'lampostfitfl', 'lamfl', 'lampostfitsig']
    
    flight_variables = [lamconflsig]
    flight_tags = ['lampostfitsig']

    for cutvariable, flight_tag in zip(flight_variables, flight_tags):
        ####### Do the study with many cuts
        nrows = 8
        ncols = 8
        
        fig1,axes1 = plt.subplots(figsize=(12,8),nrows=nrows, ncols=ncols, sharex=True, sharey=True)
        fig2,axes2 = plt.subplots(figsize=(12,8),nrows=nrows, ncols=ncols, sharex=True, sharey=True,)
    
        
        for i in range(0,60):
    
            # Cut on our flight-length variable
            cut = i*0.1
            if flight_tag=='lampostfitsig':
                cut = i*1
                        
            mask_fl = cutvariable>cut
            
            m = lamuncmass
            mask = (m>lo) & (m<hi) & mask_fl
            
            mask_lo_sideband = (m<=lo) & (m>(lo-2*width)) & mask_fl
            mask_hi_sideband = (m>=hi) & (m<(hi+2*width)) & mask_fl
    
            # Have to cut out the duplicates first
            nall = len(ak.flatten(m[lamfl_basic>=0], axis=None))

            #print(m)
            #print(mask)
            #print(m[mask])
            #xtmp = ak.flatten(m[mask], axis=None)
            #print(xtmp)
            #print(len(m), len(mask))
            
            npeak = len(ak.flatten(m[mask], axis=None))
            nbkglo = len(ak.flatten(m[mask_lo_sideband], axis=None))
            nbkghi = len(ak.flatten(m[mask_hi_sideband], axis=None))
            nbkg = (nbkglo + nbkghi)/2.0
            nsig = npeak - nbkg
                         
            print(f"nall: {nall}     npeak: {npeak}   nbkg: {nbkg}     nsig: {nsig}   {nbkglo}   {nbkghi}")
            peaks.append(npeak)
            sigs.append(nsig)
            bkgs.append(nbkg)
            cuts.append(cut)
            allentries.append(nall)
            tags.append(flight_tag)
    
            # For the different subplots
            row = int(i/ncols)
            col = i%ncols
            
            plt.sca(axes1[row][col])
            
            plt.hist(ak.flatten(m, axis=None),      bins=100,range=(1.105, 1.125))
            plt.hist(ak.flatten(m[mask], axis=None),bins=100,range=(1.105, 1.125), label=f'flcut>{cut:.1f}')
            plt.hist(ak.flatten(m[mask_lo_sideband], axis=None),bins=100,range=(1.105, 1.125), color='yellow')
            plt.hist(ak.flatten(m[mask_hi_sideband], axis=None),bins=100,range=(1.105, 1.125), color='yellow')
            plt.legend()
            
            #fig2.add_subplot(3,4,i+1)
            plt.sca(axes2[row][col])
            if row == nrows-1:
                axes1[row][col].set_xlabel(r'$\Lambda^0$ mass [GeV/c$^2$]')
                axes2[row][col].set_xlabel(r'$\Lambda^0$ flight len [cm]')

            fllo, flhi = 0,30
            if flight_tag=='lampostfitsig':
                fllo, flhi = 0,100

            plt.hist(ak.flatten(cutvariable, axis=None),         bins=100,range=(fllo, flhi))
            plt.hist(ak.flatten(cutvariable[mask_fl], axis=None),bins=100,range=(fllo, flhi))
    
        fig1.subplots_adjust(wspace=0, hspace=0)#left=0.1, right=0.1, bottom=0.1, top=0.1, wspace=0.1, hspace=0.1)#wspace=0, hspace=0)
        fig2.subplots_adjust(wspace=0, hspace=0)#left=0.1, right=0.1, bottom=0.1, top=0.1, wspace=0.1, hspace=0.1)#wspace=0, hspace=0)
    
        #fig1.subplots_adjust(0,0,1,1,0,0)
        #fig2.subplots_adjust(0,0,1,1,0,0)

        plt.figure(fig1)
        plt.suptitle(f'{flight_tag}')
        plt.figure(fig2)
        plt.suptitle(f'{flight_tag}')
    
        fig1.tight_layout()
        fig2.tight_layout()
        
    
        outfile = f'{savedir}/STUDY_flight_len_effect_on_Lambda_purity_Lambda_masses_with_cuts_{flight_tag}_{spmode}.png'
        fig1.savefig(outfile)
        outfile = f'{savedir}/STUDY_flight_len_effect_on_Lambda_purity_Lambda_flightlens_with_cuts_{flight_tag}_{spmode}.png'
        fig2.savefig(outfile)
        
        print()
        print(f"Cutting around the peak alone reduces the number of signal to {100*peaks[0]/nall:.2f} %\n\n")

    sigs = np.array(sigs)
    bkgs = np.array(bkgs)
    peaks = np.array(peaks)
    cuts = np.array(cuts)
    allentries = np.array(allentries)

    pcts = sigs/sigs[0]
    bkg_under_peak = peaks - sigs
    pct_bkg_under_peak = bkg_under_peak / bkg_under_peak[0]

    df_dict = {}
    df_dict['sigs'] = sigs
    df_dict['bkgs'] = bkgs
    df_dict['peaks'] = peaks
    df_dict['cuts'] = cuts
    df_dict['tags'] = tags
    df_dict['allentries'] = allentries
    df_dict['pcts'] = pcts
    df_dict['bkg_under_peak'] = bkg_under_peak
    df_dict['pct_bkg_under_peak'] = pct_bkg_under_peak

    df = pd.DataFrame.from_dict(df_dict)

    df['spmode'] = spmode

    return df


In [ ]:
df_998 = study_flight_length_effect_on_lambda0_purity(data, spmode='998')


In [ ]:
df_n999 = study_flight_length_effect_on_lambda0_purity(data, spmode='-999')


In [ ]:
df_1005 = study_flight_length_effect_on_lambda0_purity(data, spmode='1005')


In [ ]:

###################################################################################
def summarize_lambda_purity_studies(df, label='DEFAULT', tag='DEFAULT', units=' (cm)'):

    cuts = df['cuts'].values
    peaks = df['peaks'].values
    sigs = df['sigs'].values
    bkgs = df['bkgs'].values
    bkg_under_peak = df['bkg_under_peak'].values
    pct_bkg_under_peak = df['pct_bkg_under_peak'].values
    pcts = df['pcts'].values

    #######################
    S = sigs
    B = bkg_under_peak
    scale = 1
    #fom = 10*sigs/np.sqrt(10*bkg_under_peak)
    fom = scale*S/np.sqrt(scale*S + scale*B)
    
    max_fom = max(fom)
    #print(max_fom) 
    #idx_mask = fom== max_fom
    #print(df.loc[idx_mask])

    idx_max = fom.tolist().index(max_fom)

    print(f"idx_max: {idx_max}  max cuts: {cuts[idx_max]:.2f} max fom: {fom[idx_max]:.2f}   pcts: {pcts[idx_max]:.2f}")
    #######################
    
    spmode = df['spmode'].values[0]
    
    ############################################################
    # Summarize the values
    
    plt.figure(figsize=(12, 8))
    plt.subplot(3,2,1)
    plt.plot(cuts, peaks,'o')
    plt.ylabel('# under peak')
    plt.xlabel(f'Cut on flight value{units}')

    
    plt.subplot(3,2,2)
    plt.plot(cuts, sigs,'o')
    #plt.ylabel('# of signal in peak (# in peak - # est background')
    plt.ylabel('# of signal in peak')
    plt.xlabel(f'Cut on flight value{units}')

    plt.subplot(3,2,3)
    plt.plot(cuts, bkgs,'o')
    plt.ylabel('# est background')
    plt.xlabel(f'Cut on flight value{units}')
    
    plt.subplot(3,2,4)
    plt.plot(cuts, pcts,'o')
    plt.ylim(0.7,1.1)
    plt.ylabel('% sig remaining')
    plt.xlabel(f'Cut on flight value{units}')
    
    plt.subplot(3,2,5)
    plt.plot(cuts, pct_bkg_under_peak,'o')
    #plt.ylim(0.7,1.1)
    plt.ylabel('% bkg under peak')
    plt.xlabel(f'Cut on flight value{units}')
    
    # Naive significance
    # Multiply by 10 if we are only doing Run 1
    plt.subplot(3,2,6)
    #plt.plot(cuts, 10*sigs/np.sqrt(10*bkg_under_peak),'o')
    plt.plot(cuts, fom,'o')
    plt.ylabel(r'$S/\sqrt{ S + B }$')
    plt.xlabel(f'Cut on flight value{units}')

    plt.suptitle(f'{label}')

    plt.tight_layout()
    
    outfile = f'{savedir}/STUDY_flight_len_effect_on_Lambda_purity_summaries_{spmode}_{tag}.png'
    plt.savefig(outfile)


    return 0


In [ ]:
df_998.columns
df_998['tags'].unique()

In [ ]:
labels = {}
labels['blamtrans'] = f'$B-\Lambda^0$ transverse flight length'
labels['lampostfitfl'] = f'$\Lambda^0$ post-fit flight length'
labels['lamfl'] = f'$\Lambda^0$ flight length'
labels['lampostfitsig'] = f'$\Lambda^0$ flight length significance'

for tag in df_998['tags'].unique():

    print(tag)

    units = ' (cm)'
    if tag=='lampostfitsig':
        units = ''
    
    mask = df_998['tags'] == tag
    retval = summarize_lambda_purity_studies(df_998[mask], tag=tag, label=labels[tag], units=units)


In [ ]:
df_998.columns

In [ ]:
mask = df_998['tags'] == tag

x = df_998[mask]['sigs'].max()
print(x)

idx_mask = df_998[mask]['sigs']== x

df_998[mask].loc[idx_mask]

In [ ]:
for tag in df_n999['tags'].unique():

    print(tag)

    print(tag)

    units = ' (cm)'
    if tag=='lampostfitsig':
        units = ''
        
    mask = df_n999['tags'] == tag
    retval = summarize_lambda_purity_studies(df_n999[mask], tag=tag, label=labels[tag], units=units)


In [ ]:
for tag in df_1005['tags'].unique():

    print(tag)
    print(tag)

    units = ' (cm)'
    if tag=='lampostfitsig':
        units = ''
    
    mask = df_1005['tags'] == tag
    retval = summarize_lambda_purity_studies(df_1005[mask], tag=tag, label=labels[tag], units=units)


In [ ]:
x = ak.flatten(data['Lambda0VtxStatus'])

plt.hist(x,bins=100)
plt.yscale('log')

In [ ]:
# Does it matter if we make the PID cuts? 
###################################
# SP
###################################

#splist= np.unique(sp.to_list())
#print(splist)

dcuts= bat.get_final_masks(data, region_definitions= region_definitions)

print([dcuts.keys()])
print()

for key in dcuts.keys():
    print(f'{key:3d} {dcuts[key]["name"]}')


###################################
# Collision
###################################

dcuts_col= bat.get_final_masks(data_collision, region_definitions= region_definitions)


#bkg_spmodes= ["998","1005","3981","1235","1237"]
#sig_spmodes= ["-999"]

#spmodes= bkg_spmodes+sig_spmodes
#
#weights= {}
#for sp in spmodes: 
#    weights[sp]= bat.scaling_value(int(sp),dataset_information=dataset_information, cs_data= cs_data, plot= False, verbose= False)
#    #weights[sp]=1

#weights["-999"]= 1000 #scales signal higher 
#weights["0"]= 1 #idk what this is for;;; ASK

#print(weights)

In [ ]:
#mask = dcuts[2]['event'] & dcuts[4]['event']
mask_bool_proton, mask_bool_pion, mask_bool_protonB = bat.PID_masks(data, \
                  lamp_selector='VeryLooseKMProtonSelection', \
                  lampi_selector='VeryLooseKMPionMicroSelection', \
                  Bp_selector='SuperLooseKMProtonSelection', \
                  verbosity=0)

# We can do this because we selected the data where there is only one candidate for th
# B and Lambda0
mask_pid =      mask_bool_proton & \
                mask_bool_pion# & \
                #mask_bool_protonB

mask_event_pid = ak.any(mask_pid, axis=-1)

# Munge the shape to ensure it is the same shape as the original data array
#mask_event_pid = bat.munge_mask_shapes(mask_event_duplicates, mask_event_pid)


df_998_pid_cuts = study_flight_length_effect_on_lambda0_purity(data[mask_event_pid], spmode='998')


In [ ]:
for tag in df_998_pid_cuts['tags'].unique():

    print(tag)
    
    mask = df_998_pid_cuts['tags'] == tag
    retval = summarize_lambda_purity_studies(df_998_pid_cuts[mask], tag=labels[tag])


In [ ]:
S = df_998_pid_cuts['sigs']
B = df_998_pid_cuts['bkgs']

fom = S/np.sqrt(S+B)

df_998_pid_cuts['fom'] = fom

df_998_pid_cuts

In [ ]:
df_1005_pid_cuts = study_flight_length_effect_on_lambda0_purity(data[mask_event_pid], spmode='1005')


In [ ]:
for tag in df_1005_pid_cuts['tags'].unique():

    print(tag)
    
    mask = df_1005_pid_cuts['tags'] == tag
    retval = summarize_lambda_purity_studies(df_1005_pid_cuts[mask], tag=labels[tag])


In [ ]:
'''
for f in data.fields:
    if f.find('Lambda0')>=0:
        print(f)
'''

In [ ]:
mask_sp = data['spmode']=='-999'

fl = data['Lambda0FlightLen']
flps = data['Lambda0postFitFlight']

flerr = data['Lambda0FlightErr']
flsig = data['Lambda0postFitFlightSignificance']

plt.figure(figsize=(8,8))
plt.subplot(2,2,1)
plt.hist(ak.flatten(fl),bins=100, range=(0,5))
plt.xlabel('Lambda0FlightLen')

plt.subplot(2,2,2)
plt.hist(ak.flatten(flps),bins=100, range=(0,5))
plt.xlabel('Lambda0postFitFlight')


plt.subplot(2,2,3)
plt.hist(ak.flatten(flerr),bins=100, range=(0,1))
plt.xlabel('Lambda0FlightErr')

plt.subplot(2,2,4)
plt.hist(ak.flatten(flsig),bins=100, range=(0,100))
plt.xlabel('Lambda0postFitFlightSignificance')

;

In [ ]:
for f in data.fields:
    if f.find('Lambda')>=0 and f.find('light')>0:
        print(f)

print()

for f in data.fields:
    if f.find('Lambda')>=0 and f.find('en')>0:
        print(f)

print()

for f in data.fields:
    if f.find('Lambda')>=0 and f.find('ig')>0:
        print(f)





In [ ]:
icount = 0
for var in ['Lambda0FlightBSLen', 'Lambda0FlightLen', 'Lambda0postFitFlight', 'Lambda0postFitFlightSignificance']:
    
    spmode = '-999'
    mask = data['spmode']==spmode

    lo,hi = 0,30
    if icount==3:
        lo,hi = 0,100

    lo1,hi1 = None,None
    if var=='Lambda0postFitFlightSignificance':
        lo1,hi1 = 0,500

    print("Signal")
    fail_mask = (data[mask][var] < -1)
    pass_mask = (data[mask][var] > -1)
    nfail = ak.sum(ak.num(fail_mask[fail_mask]))
    npass = ak.sum(ak.num(pass_mask[pass_mask]))    
    pct_fail = nfail/(npass+nfail)
    print(f"{var:40s}   pct fail: {100*pct_fail:.2f}")
    
    plt.figure(figsize=(12,8))
    plt.subplot(3,2,1)
    if var=='Lambda0postFitFlightSignificance':
        plt.hist(ak.flatten(data[mask][var]), bins=100, range=(0,500), label='MC sig')
    else:
        plt.hist(ak.flatten(data[mask][var]), bins=100, label='MC sig')
    plt.xlabel(var)
    plt.subplot(3,2,2)
    plt.hist(ak.flatten(data[mask][var]), bins=100, range=(lo,hi), label='MC sig')
    plt.xlabel(var)
    plt.legend()

    mask = data['spmode']!=spmode

    print("Background")
    fail_mask = (data[mask][var] < -1)
    pass_mask = (data[mask][var] > -1)
    nfail = ak.sum(ak.num(fail_mask[fail_mask]))
    npass = ak.sum(ak.num(pass_mask[pass_mask]))    
    pct_fail = nfail/(npass+nfail)
    print(f"{var:40s}   pct fail: {100*pct_fail:.2f}")

    plt.subplot(3,2,3)
    if var=='Lambda0postFitFlightSignificance':
        plt.hist(ak.flatten(data[mask][var]), bins=100, range=(0,500), label='MC bkg')
    else:
        plt.hist(ak.flatten(data[mask][var]), bins=100, label='MC bkg')

    #plt.hist(ak.flatten(data[mask][var]), bins=100, label='MC')
    plt.xlabel(var)
    plt.subplot(3,2,4)
    plt.hist(ak.flatten(data[mask][var]), bins=100, range=(lo,hi), label='MC bkg')
    plt.xlabel(var)
    plt.legend()


    print("Collision")
    fail_mask = (data_collision[var] < -1)
    pass_mask = (data_collision[var] > -1)
    nfail = ak.sum(ak.num(fail_mask[fail_mask]))
    npass = ak.sum(ak.num(pass_mask[pass_mask]))    
    pct_fail = nfail/(npass+nfail)
    print(f"{var:40s}   pct fail: {100*pct_fail:.2f}")
    print()

    plt.subplot(3,2,5)
    if var=='Lambda0postFitFlightSignificance':
        plt.hist(ak.flatten(data[mask][var]), bins=100, range=(0,500), label='Data')
    else:
        plt.hist(ak.flatten(data[mask][var]), bins=100, label='Data')
    #plt.hist(ak.flatten(data_collision[var]), bins=100, label='Data')
    plt.xlabel(var)
    plt.subplot(3,2,6)
    plt.hist(ak.flatten(data_collision[var]), bins=100, range=(lo,hi), label='Data')
    plt.xlabel(var)
    plt.legend()

    plt.tight_layout()
    plt.savefig(f'BNV_pLambda_plots/STUDY_Compare_MC_sig_MC_bkg_collision_data_{var}.png')
    icount += 1
;

# Cutting on angle between momentum and position

In [ ]:
def spherical_to_cartesian(p3_spher):
    r = p3_spher[0]
    costh = p3_spher[1]
    phi = p3_spher[2]
    
    #print(p3_spher)
    #print(costh)
    theta = np.arccos(costh)

    x = r*np.sin(theta)*np.cos(phi)
    y = r*np.sin(theta)*np.sin(phi)
    z = r*np.cos(theta)

    pmag = np.sqrt(x**2 + y**2 + z**2)
    print(pmag, r)

    return x,y,z, pmag

def angle_between_vectors(p3s):
  p0mag = np.sqrt(p3s[0][0]**2 + p3s[0][1]**2 + p3s[0][2]**2)
  p1mag = np.sqrt(p3s[1][0]**2 + p3s[1][1]**2 + p3s[1][2]**2)

  #print("p3s")
  #print(p3s[0][0])
  #print(p3s[1][0])
  #print(len(p3s[0][0]))
  #print(len(p3s[1][0]))

  dot_product = p3s[0][0]*p3s[1][0] + p3s[0][1]*p3s[1][1] + p3s[0][2]*p3s[1][2]

  theta = np.arccos(dot_product/(p0mag*p1mag))

  return theta

angle_between_vectors([[1, 1, 0], [1, 0, 0]])

In [ ]:
#mask = data['spmode'] == '-999'
#mask = data['spmode'] == '998'
mask = data['spmode'] == '1005'
#mask = data['spmode'] == '-1'

bvtx = data[mask]['B_con_Vtxx']
bvty = data[mask]['B_con_Vtxy']
bvtz = data[mask]['B_con_Vtxz']

lamvtx = data[mask]['Lambda0Vtxx']
lamvty = data[mask]['Lambda0Vtxy']
lamvtz = data[mask]['Lambda0Vtxz']

lamconfl =    data[mask]['Lambda0postFitFlight']
lamconflsig = data[mask]['Lambda0postFitFlightSignificance']

bconfl =    data[mask]['BpostFitFlight']
bconflsig = data[mask]['BpostFitFlightSignificance']


blamfldx = bvtx - lamvtx
blamfldy = bvty - lamvty

blamfltrans2 = blamfldx**2 + blamfldx**2
blamfltrans = np.sqrt(blamfltrans2)

lamfltrans2 = lamvtx**2 + lamvtx**2
lamfltrans = np.sqrt(lamfltrans2)


vecx = lamvtx - bvtx
vecy = lamvty - bvty
vecz = lamvtz - bvtz

p3_spher = [data[mask]['Lambda0p3'], data[mask]['Lambda0costh'], data[mask]['Lambda0phi'] ]
print(len(p3_spher))
print(p3_spher)

lampx, lampy, lampz, lampmag = spherical_to_cartesian(p3_spher)
print("---------")
print(lampx)
print(lampmag)
print(data[mask]['Lambda0p3'])
print("--------")

theta = angle_between_vectors([[lampx[:,0], lampy[:,0], lampz[:,0]], [vecx[:,0], vecy[:,0], vecz[:,0]]])

print(theta)

print(bvtx.type)
print(lamvtx.type)
print(blamfltrans.type)
print(blamfltrans.type)


lamuncmass = data[mask]['Lambda0_unc_Mass']

In [ ]:
plt.figure()
plt.hist(theta, bins=100,range=(0,.150));
plt.yscale('log')

print(len(theta))
print(len(theta[theta<0.01]))

y = data[mask]['Lambda0FlightLen'][:,0]

mask_y = (y>0)

plt.figure()
plt.hist(y[mask_y], bins=500,range=(0,50));
plt.yscale('log')
plt.xscale('log')


plt.figure()
plt.plot(theta[mask_y], y[mask_y], '.', markersize=0.5)
plt.xlim(0,0.02)
plt.ylim(0.05, 50)
plt.yscale('log')
#plt.xscale('log')

z = data[mask]['Lambda0_unc_Mass'][:,0]

#print(len(theta[mask]), len(y[mask& mask_y]), len(z[mask& mask_y]))

plt.figure()
plt.plot(theta[mask_y], z[mask_y], '.', markersize=0.5)
plt.xlim(0,0.1)
#plt.ylim(0.4)

mask_theta = (theta < 0.004)

plt.figure()
plt.hist(z[mask_y], bins=100)
plt.hist(z[mask_y & mask_theta], bins=100)

#plt.xlim(0,0.01)
#plt.ylim(0.4)
;

In [ ]:
data['Lambda0_unc_costh']

In [ ]:
for f in data.fields:
    if f.find('Vtx') >=0:
        print(f)